#### Run service:
```bash
    docker-compose up -d
```
with changes in `tasks.py` add `--build`

In [1]:
import pickle
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted


In [2]:
from example import get_results, run_task
from tasks import train

In [3]:
# data

file_path = "data/penguins.parquet"
dataf = pd.read_parquet(file_path)

FEATURES = [
    "bill_length_mm",
    "bill_depth_mm",
    "flipper_length_mm",
    "body_mass_g",
    "sex",
    "island",
]
TARGET = "species"

dataf.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
index,,,,,,,
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female


In [4]:
# pipeline
# preprocessing data
preprocessor = ColumnTransformer(
        transformers=[
            ("num", MinMaxScaler(), selector(dtype_exclude="object")),
            ("cat", OneHotEncoder(drop="first"), selector(dtype_include="object")),
        ]
    )

# classification model with defaults parameters
clf = GradientBoostingClassifier()
classifier = Pipeline(steps=[("preprocesser", preprocessor), ("GBC", clf)])

In [5]:
try:
    check_is_fitted(classifier[1], msg="Not Fitted")
except NotFittedError as e:
    print(e)

Not Fitted


In [6]:
# `sudo setfacl -m u:[username]:rwx model` if root created model
model_path = "models/model_example.pickle"
pipeline_path = "models/pipeline.pickle"
with open(pipeline_path, "wb") as f:
    pickle.dump(classifier, f)

# run task in background
d = run_task(
    func=train,
    task_name="classifer",
    model_path=model_path,
    pipeline_path=pipeline_path,
    features=FEATURES,
    target=TARGET,
    file_path=file_path,
)

task_id = d.get("task_id")
status = get_results(task_id)

In [7]:
status

{'task_id': 'a2fabf14-e822-4bb2-8fdb-325afec04583',
 'task_status': 'PENDING',
 'task_result': None}

_task is sent to background_<br>
`Do other things as you wait ...`

In [11]:
status = get_results(task_id)
status

{'task_id': 'a2fabf14-e822-4bb2-8fdb-325afec04583',
 'task_status': 'SUCCESS',
 'task_result': {'model_path': 'models/model_example.pickle'}}

In [12]:
with open(model_path, "rb") as f:
    returned_pipe = pickle.load(f)

In [13]:
print(returned_pipe)

Pipeline(steps=[('preprocesser',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f82d2c8fd30>),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f8308d32130>)])),
                ('GBC', GradientBoostingClassifier())])


In [14]:
check_is_fitted(returned_pipe[1], msg="Not Fitted")